In [1]:
import pandas as pd
import numpy as np
import tkinter as tk
import matplotlib.pyplot as plt
from tkinter import filedialog
import datetime

In [2]:
root =tk.Tk()
root.withdraw()
filepath=filedialog.askopenfilename()
filepath


'C:/Users/Rafael Pietro/OneDrive/Cursos/DNC/Datasets/Estruturação de dados/DCW 8 - sample_medicare.csv'

In [6]:
data_consumo = {
    0: {"id": "xx1", "dt_consumo": "2020-11-03", "vl_consumo": 50},
    1: {"id": "xx1", "dt_consumo": "2020-11-15", "vl_consumo": 21},
    2: {"id": "xx2", "dt_consumo": "2020-11-18", "vl_consumo": 33},
    3: {"id": "xx2", "dt_consumo": "2020-12-16", "vl_consumo": 23},
    4: {"id": "xx3", "dt_consumo": "2020-11-06", "vl_consumo": 45},
    5: {"id": "xx4", "dt_consumo": "2020-12-18", "vl_consumo": 46},
    6: {"id": "xx4", "dt_consumo": "2020-11-18", "vl_consumo": 12},
    7: {"id": "xx4", "dt_consumo": "2020-11-04", "vl_consumo": 34},
}

In [7]:
data_id = {
    0: {"id": "xx1", "dt_compra": "2020-12-01", "flg_comprou": 1},
    1: {"id": "xx2", "dt_compra": "2020-12-15", "flg_comprou": 1},
    2: {"id": "xx3", "dt_compra": np.nan, "flg_comprou": 0},
    3: {"id": "xx4", "dt_compra": np.nan, "flg_comprou": 0}
}
# Dataset com o TARGET, onde sei se eles compraram ou não.

In [8]:
df_consumo = pd.DataFrame.from_dict(data_consumo).transpose()
df_id = pd.DataFrame.from_dict(data_id).transpose()

In [10]:
df_consumo.head()

,id,dt_consumo,vl_consumo
0,xx1,2020-11-03,50
1,xx1,2020-11-15,21
2,xx2,2020-11-18,33
3,xx2,2020-12-16,23
4,xx3,2020-11-06,45


In [11]:
df_id.head()

,id,dt_compra,flg_comprou
0,xx1,2020-12-01,1
1,xx2,2020-12-15,1
2,xx3,NaN,0
3,xx4,NaN,0


In [23]:
df_merge=pd.merge(df_id,df_consumo,left_on="id",right_on="id",how="left")
df_merge

,id,dt_compra,flg_comprou,dt_consumo,vl_consumo
0,xx1,2020-12-01,1,2020-11-03,50
1,xx1,2020-12-01,1,2020-11-15,21
2,xx2,2020-12-15,1,2020-11-18,33
3,xx2,2020-12-15,1,2020-12-16,23
4,xx3,NaN,0,2020-11-06,45
5,xx4,NaN,0,2020-12-18,46
6,xx4,NaN,0,2020-11-18,12
7,xx4,NaN,0,2020-11-04,34


In [29]:
df_merge.loc[df_merge.dt_compra.isna(),"dt_compra"]="2020-12-15"
df_merge
# Estamos fazendo isso só para preencher os vazios pq era a data de corte da compra.
# Ou seja, só estamos querendo pessoas que compraram ATÉ dia 15/12.
# Estamos trabalhando no passado (até dia 15/12, por isso tiraremos os demais)

,id,dt_compra,flg_comprou,dt_consumo,vl_consumo
0,xx1,2020-12-01,1,2020-11-03,50
1,xx1,2020-12-01,1,2020-11-15,21
2,xx2,2020-12-15,1,2020-11-18,33
3,xx2,2020-12-15,1,2020-12-16,23
4,xx3,2020-12-15,0,2020-11-06,45
5,xx4,2020-12-15,0,2020-12-18,46
6,xx4,2020-12-15,0,2020-11-18,12
7,xx4,2020-12-15,0,2020-11-04,34


In [37]:
df_select=df_merge.loc[df_merge.dt_consumo<df_merge.dt_compra,:]
df_select
# estamos tirando dados sobre o "futuro", como se hoje fosse dia 15/12.

,id,dt_compra,flg_comprou,dt_consumo,vl_consumo
0,xx1,2020-12-01,1,2020-11-03,50
1,xx1,2020-12-01,1,2020-11-15,21
2,xx2,2020-12-15,1,2020-11-18,33
4,xx3,2020-12-15,0,2020-11-06,45
6,xx4,2020-12-15,0,2020-11-18,12
7,xx4,2020-12-15,0,2020-11-04,34


In [41]:
df_select[["id","vl_consumo"]].groupby(by="id").sum().reset_index()

,id,vl_consumo
0,xx1,71
1,xx2,33
2,xx3,45
3,xx4,46


In [42]:
df_select[["id","vl_consumo"]].groupby(by="id").count().reset_index()

,id,vl_consumo
0,xx1,2
1,xx2,1
2,xx3,1
3,xx4,2


In [52]:
df_agg = df_select[['id', 'dt_consumo', 'vl_consumo']].groupby(by="id").agg({
    'dt_consumo': 'max',
    'vl_consumo': ['sum','min','mean','max'],
    'id': 'count'
})
df_agg

dt_consumo vl_consumo                  id
            max        sum min  mean max count
id                                            
xx1  2020-11-15         71  21  35.5  50     2
xx2  2020-11-18         33  33  33.0  33     1
xx3  2020-11-06         45  45  45.0  45     1
xx4  2020-11-18         46  12  23.0  34     2

In [53]:
df_agg.columns = ['.'.join(col).strip() if col[1] != "" else col[0] for col in df_agg.columns.values]

In [57]:
df_agg.reset_index()

,id,dt_consumo.max,vl_consumo.sum,vl_consumo.min,vl_consumo.mean,vl_consumo.max,id.count
0,xx1,2020-11-15,71,21,35.5,50,2
1,xx2,2020-11-18,33,33,33.0,33,1
2,xx3,2020-11-06,45,45,45.0,45,1
3,xx4,2020-11-18,46,12,23.0,34,2
